# 목차
1. 파이프라인 개요 및 목표
2. 데이터 로드 및 전처리
3. 자동 특징 공학
    - FeatureTools
    - tsfresh
    - tslearn
4. 데이터 증강
    - tsaug 활용
    - 고급 데이터 증강 기법
5. 이벤트 기반 특징 추출 및 증강
6. 모델 하이퍼파라미터 튜닝
    - Optuna 활용
7. 다양한 모델 학습 및 이상치 탐지
    - Autoencoder
    - Variational Autoencoder(VAE)
    - Generative Adversarial Networks (GANs)
    - Deep Belief Networks (DBN)
    - LSTM Autoencoder
    - Transformer Autoencoder
8. 앙상블 방법: Majority Voting
9. 모델 평가
10. 모델 해석 및 투명성 강화
    - SHAP
    - LIME
11. 모델 최적화 및 경량화
    - ONNX 변환
    - TensorRT 최적화
12. 파이프라인 자동화: Airflow활용
13. 모델 배포 및 서비스화
    - Flask를 사용한 REST API 구축
14. 결론 및 추가 제안

***
## 1. 파이프라인 개요 및 목표

>RiskPrediction 파이프라인은 시계열 데이터를 기반으로 이상치를 탐지하는 시스템을 구축하는 것을 목표로 함.
>이 파이프라인은 데이터 로드부터 전처리, 특징 공학, 모델 학습, 이상치 탐지, 모델 해석, 최적화, 자동화, 배포에 이르는 전 과정을 포함

- **데이터의 효율적 처리**: 대용량 시계열 데이터를 효과적으로 로드하고 전처리함
- **자동 특징 추출**: 다양한 라이브러리(FeatureTools, tsfresh, tslearn)를 활용하여 데이터에서 유의미한 특징을 자동으로 추출함
- **데이터 증강**: tsaug를 사용하여 데이터의 다양성을 높이고 모델의 일반화 능력을 향상 시킴
- **다양한 모델 학습**: Autoencoder, VAE, GANs, DBN, LSTM_AE, Transformer_AE 등 여러 비지도 학습 모델을 학습시켜 이상치를 탐지함
- **앙상블 방법 도입**: Majority Voting을 통해 여러 모델의 예측을 결합하여 최종 이상치 판별을 수행함
- **모델 해석**: SHAP과 LIME을 사용하여 모델의 예측을 해석하고 투명성을 높임
- **모델 최적화**: ONNX와 TensorRT를 활용하여 모델의 추론 속도를 최적화하고 경량화함
- **파이프라인 자동화**: Airflow를 사용하여 데이터 처리, 모델 학습, 평가, 배포 등의 단계를 자동화함
- **모델 배포**: Flask를 사용하여 학습된 모델을 REST API 형태로 배포하여 실시간 예측 시스템을 구축

***
## 2. 데이터 로드 및 전처리

> 데이터 로드 및 전처리 단계에서는 시계열 데이터를 효율적으로 로드하고, 결측치를 처리하며, 필요한 전처리 작업을 수행함.
> 기본적인 특징 공학을 통해 모델 학습에 필요한 기초적인 특징을 생성함

### a. 필요한 라이브러리 설치

```
%pip install dask[complete] scikit-learn featuretools tsfresh tslearn tsaug optuna airflow flask shap lime pandas numpy matplotlib seaborn torch torchvision
```

### b. 데이터 로드 함수 구현

> Dask를 사용하여 대용량 CSV 데이터를 효율적으로 로드하고, 필요한 전처리 작업을 수행

In [1]:
import logging
import sys
import pandas as pd
import numpy as np
import dask.dataframe as dd
import featuretools as ft
import shap
from lime import lime_tabular
import tensorrt as trt

from dask.diagnostics import ProgressBar
from sklearn.preprocessing import StandardScaler
from tsfresh import extract_features
# from tslearn.feature_extraction import FeatureRep
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tsaug import TimeWarp, Drift, AddNoise, Crop, Quantize, Reverse

import optuna
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import silhouette_score, devies_bouldin_score, calinski_harabasz_score

import torch
import torch.onnx
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# 로깅 설정
logging.basicConfig(
    level = logging.INFO,
    format = '%(asctime)s [%(levelname)s] %(message)s',
    handlers = [
        logging.FileHandler("./Log/RiskPrediction.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger()

def load_and_preprocess_data(data_file, sample_frace=0.1, window_size=100):
    """
    데이터 로드 및 전처리 함수
    - data_file: 데이터 파일 경로
    - sample_frace: 샘플링 비율
    - window_size: 윈도우 크기 (이벤트 기반 특징 추출용)
    """

    try:
        logger.info("Dask를 사용하여 데이터 로드 시작")
        with ProgressBar():
            df = dd.read_csv(data_file, parse_dates=['VITALDATE', 'CHECKTIME'], assume_missing=True)
            logger.info(f"데이터 로드 완료: {df.shape[0].compute()} rows, {df.shape[1].compute()} columns")
    except Exception as e:
        logger.error(f"데이터 로드 중 오류 발생: {e}")
        sys.exit(1)

    try:
        # 필요한 열만 선택
        selected_columns = ['VITALDATE',
                            'HEARTBEAT',
                            'TEMPERATURE',
                            'OUTSIDETEMPERATURE', 
                            'LATITUDE',
                            'LONGITUDE',
                            'SPEED',
                            'X',
                            'Y',
                            'Z', 
                            'Event']  # 'Event' 컬럼 추가
        df = df[selected_columns]

        # 결측치 제거
        logger.info("결측치 제거 시작")
        df = df.dropna()
        logger.info(f"결측치 제거 완료: {df.shape[0].compute()} rows 남음")

        # 데이터 타입 변환 (float형으로)
        numeric_cols = ['HEARTBEAT',
                        'TEMPERATURE',
                        'OUTSIDETEMPERATURE', 
                        'LATITUDE',
                        'LONGITUDE',
                        'SPEED',
                        'X', 
                        'Y', 
                        'Z']
        df[numeric_cols] = df[numeric_cols].astype(float)

        # 정규화 (표준화)
        logger.info("데이터 정규화 시작")
        scaler = StandardScaler()
        df[numeric_cols] = dd.from_array(scaler.fit_transform(df[numeric_cols].compute()), columns=numeric_cols)
        logger.info("데이터 정규화 완료")

        # 기본 특징 공학
        logger.info("기본 특징 공학 시작")

        # 심박수에서 RR 간격 추정 (초 단위)
        df['RR_interval'] = 60 / df['HEARTBEAT']

        # RR_interval의 결측치 제거
        df = df.dropna(subset=['RR_interval'])

        # 시간순으로 정렬
        df = df.map_partitions(lambda partition: partition.sort_values('VITALDATE'))

        # NN 간격 계산 (연속하는 RR_interval 간의 차이)
        df['NN_interval'] = df['RR_interval'].diff()

        # SDNN (NN_interval의 표준편차)
        SDNN = df['NN_interval'].std().compute()
        logger.info(f"SDNN: {SDNN}")

        # RMSSD (NN_interval의 제곱 평균 제곱근)
        RMSSD = np.sqrt((df['NN_interval'] ** 2).mean().compute())
        logger.info(f"RMSSD: {RMSSD}")

        # 자이로스코프 데이터에서 회전 속도 변화 측정
        df['Gyro_magnitude'] = np.sqrt(df['X'] ** 2 + df['Y'] ** 2 + df['Z'] ** 2)

        # 회전 속도 변화의 변동성 측정 (Rolling Standard Deviation)
        df['Gyro_std'] = df['Gyro_magnitude'].rolling(window=window_size).std()

        # 온도 차이 계산
        df['Temperature_difference'] = df['TEMPERATURE'] - df['OUTSIDETEMPERATURE']

        # 시간 기반 특징 생성
        # 시간대 (시간)
        df['Hour'] = df['VITALDATE'].dt.hour

        # 작업 시작 후 경과 시간 (첫 번째 기록 시각으로부터의 시간 차이)
        start_time = df['VITALDATE'].min().compute()
        df['Time_since_start'] = (df['VITALDATE'] - start_time).dt.total_seconds() / 3600  # 시간 단위

        logger.info("기본 특징 공학 완료")

        return df, scaler, numeric_cols, window_size
    except Exception as e:
        logger.error(f"데이터 전처리 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
1. **데이터로드**: Dask를 사용하여 대용량 CSV 데이터를 효율적으로 로드
2. **필요한 열 선택**: 분석에 필요한 열만 선택하여 메모리 사용을 최적화
3. **결측치 제거**: 결측치를 제거하여 데이터의 품질을 향상 시킴
4. **데이터 타입 변환 및 정규화**: 수치형 데이터를 float으로 변환하고, StandardScaler를 사용하여 정규화함
5. **기본 특징 공학**:
    - **RR 간격 계산**: 심박수에서 RR 간격을 추정
    - **NN 간격 및 SDNN, RMSSD 계산**: 심박수 변동성을 측정하는 지표를 계산
    - **자이로스코프 데이터 처리**: 자이로스코프 데이터를 기반으로 회전 속도 변화의 변동성을 측정
    - **온도 차이 계산**: 내부 및 외부 온도 차이를 계산하여 환경 변화의 영향을 파악함
    - **시간 기반 특징 생성**: 시간대 및 작업 시작 후 경과 시간을 계산하여 시간에 따른 패턴 변화를 포착함

***
## 3. 자동 특징 공학

> 자동 특징 공학은 데이터에서 유의미한 특징을 자동으로 추출하는 과정.
> 이를 통해 도메인 전문가의 개입 없이도 모델의 성능을 향상시킬 수 있음.
> 여기서는 **FeatureTools, tsfresh, tslearn 을 활용하여 다양한 특징을 추출함

### a. FeatureTools 활용

> **FeatureTools는 자동화된 특징 공학을 지원하는 라이브러리로, 복잡한 데이터 구조에서도 효율적으로 특징을 생성할 수 있음

In [2]:
def auto_feature_engineering(df):
    """
    FeatureTools를 사용하여 자동 특징 공학을 수행함
    - df: pandas DataFrame
    """
    try:
        logger.info("FeatureTools를 사용한 자동 특징 공학 시작")

        # EntitySet 생성
        es = ft.EntitySet(id="risk_prediction")

        # 인덱스 설정
        df = df.reset_index()
        es = es.add_dataframe(
            dataframe_name = "data",
            dataframe = df,
            index = "index",
            time_index = "VITALDATE"
        )

        # 트랜스폼 프리미티브 설정
        trans_primitives = ["add", "multiply", "subtract", "divide", "abs", "sqrt"]

        # 특징 생성
        feature_matrix_ft, feature_defs_ft = ft.dfs(
            entityset = es,
            target_dataframe_name = "data",
            trans_primitives = trans_primitives,
            max_depth = 2,
            features_only = False,
            verbose = True
        )

        logger.info(f"FeatureTools를 사용한 자동 특징 공학 완료: 생성된 특징 수 {feature_matrix_ft.shape[1]}")
        return feature_matrix_ft
    except Exception as e:
        logger.error(f"FeatureTools를 사용한 자동 특징 공학 중 오류 발생: {e}")
        return None

### 설명:
- **EntitySet 생성**: 데이터프레임을 FeatureTools의 EntitySet에 추가하여 특징 공학을 수행할 수 있는 환경을 조성함
- **트랜스폼 프리미티브 설정**: 기본적인 수학 연산을 통해 새로운 특징을 생성함
- **특징 생성**: `ft.dfs` 함수를 사용하여 지정된 트랜스폼 프리미티브를 기반으로 자동으로 특징을 생성함

### b.tsfresh 활용

> **tsfresh**는 시계열 데이터에서 유의미한 특징을 자동으로 추출하는 라이브러리로, 다양한 시계열 통계적 특징을 제공함

In [3]:
def extract_tsfresh_features(df):
    """
    tsfresh를 사용하여 시계열 데이터의 특징을 추출함
    - df: pandas DataFrame
    """
    try:
        logger.info("tsfresh를 사용한 특징 추출 시작")

        # 시계열 데이터의 포맷 변환
        ts_data = df[['index', 'VITALDATE', 'HEARTBEAT', 'TEMPERATURE', 'OUTSIDETEMPERATURE', 
                     'LATITUDE', 'LONGITUDE', 'SPEED', 'X', 'Y', 'Z']]

        # 특징 추출
        feature_matrix_tsfresh = extract_features(ts_data, column_id = "index", column_sort = "VITALDATE")

        logger.info(f"tsfresh를 사용한 특징 추출 완료: 생성된 특징 수 {feature_matrix_tsfresh.shape[1]}")

    except Exception as e:
        logger.error(f"tsfresh를 사용한 특징 추출 중 오류 발생: {e}")
        return None

### 설명:
- **시계열 데이터 포맷 변환**: tsfresh는 특정 포맷의 시계열 데이터를 필요로 하므로, 데이터프레임을 적절히 변환함
- **특징 추출**: `extract_features` 함수를 사용하여 다양한 시계열 통계적 특징을 자동으로 추출함

### c. tslearn 활용

> **tslearn**은 시계열 데이터 분석을 위한 다양한 도구를 제공하는 라이브러리로, 시계열 데이터의 패턴을 효과적으로 추출할 수 있음

In [4]:
def extract_tslearn_features(X, n_features=10):
    """
    tslearn을 사용하여 시계열 데이터의 특징을 추출함
    - X: 입력 데이터 (numpy array), shape (samples, features)
    - n_features: 추출할 특징 수
    """
    try:
        logger.info("tslearn을 사용한 특징 추출 시작")

        # 시계열 스케일링
        scaler = TimeSeriesScalerMeanVariance()
        X_scaled = scaler.fit_transform(X)

        # FeatureRep를 사용한 특징 추출 (예: Fast Fourier Transform 등)
        feature_rep = FeatureRep(n_coefficients=n_features, random_state=42)
        X_tslearn = feature_rep.fit_transform(X_scaled)

        logger.info(f"tslearn을 사용한 특징 추출 완료: 추출된 특징 수 {X_tslearn.shape[1]}")
        return X_tslearn
    except Exception as e:
        logger.error(f"tslearn을 사용한 특징 추출 중 오류 발생: {e}")
        return None

### 설명:
- **시계열 스케일링**: 시계열 데이터를 평균과 분산을 기준으로 정규화하여 특징 추출의 효율성을 높임
- **FeatureRep 활용**: 다양한 시계열 변환 기법(Fast Fourier Transform 등)을 통해 특징을 추출함

***
## 4. 데이터 증강

> 데이터 증강은 모델의 일반화 능력을 향상시키기 위해 데이터의 다양성을 높이는 기법.   
> 시계열 데이터의 경우, 다양한 증강 기법을 통해 패턴의 변화를 생성할 수 있음   
> 여기서는 **tsauf** 라이브러리를 활용하여 고급 데이터 증강을 수행   
> **Time Warping, Window Slicing, Permutation, Magnitude Scaling, Jittering**

In [5]:
def advanced_data_augmentation(X, n_augment=2):
    """
    고급 시계열 데이터 증강 함수
    - X: 입력 데이터 (numpy array), shape (samples, features)
    - n_augment: 각 샘플당 증강할 데이터 수
    """
    try:
        logger.info(f"고급 시계열 데이터 증강 시작: n_augment = {n_augment}")

        # 다양한 증강 기법 정의
        augmenter = (
            TimeWarp(n_speed_change=2) * 2 +
            Drift(scale=0.05) +
            Scale(scale=(0.95, 1.05)) +
            AddNoise(scale=0.02) +
            Shuffle(n_shuffle=2) +
            Stretch(scale=(0.8, 1.2)) +
            Quantize(n_levels=8) + 
            Reverse()
        )

        # 증강 데이터 생성
        augmented_data = []
        for _ in range(n_augment):
            augmented = augmenter.augment(X)
            augmented_data.append(augmented)
        X_augmented = np.concatenate(augmented_data, axis=0)

        logger.info(f"고급 시계열 데이터 증강 완료: 증강된 데이터 수 {X_augmented.shape[0]}")
        return X_augmented
    except Exception as e:
        logger.error(f"고급 시계열 데이터 증강 중 오류 발생: {e}")
        return X

### 설명:
- **증강 기법 정의**:
    - **Time Warping**: 시계열 의 시간 축을 변형하여 패턴의 변화를 만듦
    - **Drift**: 데이터의 기울기를 변경하여 다양한 변화를 만듦
    - **Scale**: 데이터의 크기를 변경하여 다양한 규모의 패턴을 만듦
    - **AddNoise**: 데이터에 노이즈를 추가하여 모델의 견고성을 높임
    - **Shuffle**: 시계열 데이터의 일부를 섞어 새로운 패턴을 만듦
    - **Stretch**: 시계열의 길이를 늘이거나 줄여 패턴의 변화를 만듦
    - **Quantize**: 시계열 데이터를 양자화하여 패턴의 단순화를 유도함
    - **Reverse**: 시계열 데이터를 반전시켜 패턴의 방향성을 변화시킴
- **증강 데이터 생성**: 지정된 횟수만큼 데이터를 증강하여 원본 데이터의 다양성을 높임

***
## 5. 이벤트 기반 특징 추출 및 증강

> 원본 데이터가 이벤트 기반성이 강한 경우, 이벤트의 발생 빈도, 간격, 지속 시간 등의 특징을 추출하고 이를 증강에 반영하는 것이 중요   
> 이를 통해 모델이 이벤트의 패턴을 효과적으로 학습할 수 있음

### a. 이벤트 기반 특징 추출

> 이벤트의 발생 패턴을 분석하기 위해 이벤트 기반 특징을 추출함

In [6]:
def extract_event_features(df, event_column='Event'):
    """
    이벤트 기반 데이터의 특징을 추출함
    - df: pandas DataFrame
    - event_column: 이벤트를 나타내는 컬럼 이름
    """
    try:
        logger.info("이벤트 기반 특징 추출 시작")

        # 이벤트 발생 여부를 나타내는 이진 변수 생성
        df['event_flag'] = df[event_column].apply(lambda x: 1 if x else 0)

        # 이벤트 발생 간격 계산
        df['event_diff'] = df['event_flag'].diff().fillna(0)

        # 이벤트 간격의 평균, 표준편차
        event_intervals = df['event_diff'].replace(0, np.nan).dropna()
        df['event_interval_mean'] = event_intervals.mean()
        df['event_interval_std'] = event_intervals.std()

        # 이벤트 지속 시간 계산 (rolling window)
        df['event_duration'] = df['event_flag'].rolling(window=window_size).sum()

        logger.info("이벤트 기반 특징 추출 완료")
        return df
    except Exception as e:
        logger.error(f"이벤트 기반 특징 추출 중 오류 발생: {e}")
        return df

### 설명:
- **event_flag**: 특정 시점에 이벤트가 발생했는지 여부를 이진 변수로 나타냄
- **event_diff**: 이벤트 발생 간의 간격을 계산하여 이벤트 간의 변화 패턴을 파악함
- **event_interval_mean, event_interval_std**: 이벤트 발생 간격의 평균과 표준편차를 계산하여 이벤트의 빈도 및 변동성을 측정함
- **event_duration**: 일정 윈도우 내 이벤트의 지속 시간을 계산하여 이벤트의 지속성을 파악함

### b. 이벤트 기반 데이터 증강

> 이벤트 기반 특징을 유지하면서 데이터를 증강함.
> 예를 들어, 이벤트의 발생 시점을 변경하거나, 이벤트의 지속 시간을 변형시킬 수 있음

In [7]:
def event_based_data_augmentation(df, n_augment=2):
    """
    이벤트 기반 시계열 데이터 증강 함수
    - df: pandas DataFrame
    - n_augment: 각 샘플당 증강할 데이터 수
    """
    try:
        logger.info(f"이벤트 기반 시계열 데이터 증강 시작: n_augment = {n_augment}")

        augmented_dfs = []
        for _ in range(n_augment):
            df_aug = df.copy()

            # 이벤트 발생 시점 랜덤 변경
            event_shift = np.random.randint(-5, 6)  # 예: -5에서 +5 샘플 이동
            df_aug['event_flag'] = df_aug['event_flag'].shift(event_shift).fillna(0)

            # 이벤트 지속 시간 변형
            duration_scaling = np.random.uniform(0.8, 1.2)
            df_aug['event_duration'] = (df_aug['event_duration'] * duration_scaling).astype(int)

            augmented_dfs.append(df_aug)

        df_augmented = pd.concat(augmented_dfs, ignore_index=True)
        logger.info(f"이벤트 기반 시계열 데이터 증강 완료: 증강된 데이터 수 {df_augmented.shape[0]}")
        return df_augmented
    except Exception as e:
        logger.info(f"이벤트 기반 시계열 데이터 증강 중 오류 발생: {e}")
        return df

### 설명:
- **이벤트 시점 변경**: 이벤트 발생 시점을 랜덤하게 변경하여 다양한 이벤트 패턴을 생성함
- **이벤트 지속 시간 변형**: 이벤트의 지속 시간을 변형시켜 모델이 다양한 이벤트 지속성을 학습할 수 있도록 함

***
## 6. 모델 하이퍼파라미터 튜닝

> 모델의 성능을 최적화하기 위해 하이퍼파라미터 튜닝을 수행함.
> **Optuna**를 활용하여 자동화된 하이퍼파라미터 최적화를 진행함

### a. Optuna 활용

> **Optuna**는 효율적인 하이퍼파라미터 최적화를 지원하는 라이브러리로, 다양한 최적화 알고리즘을 제공함.

In [8]:
def objective_iso(trial, X):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    contamination = trial.suggest_float('contamination', 0.001, 0.002)
    max_samples = trial.suggest_categorical('max_samples', ['auto', 0.5, 0.75, 1.0])

    model = IsolationForest(n_estimators = n_estimators,
                            contamination = contamination,
                            max_samples = max_samples,
                            random_state = 42)
    model.fit(X)
    labels = pd.Series(model.predict(X)).map({1: 0, -1: 1})  # 0: 정상, 1: 이상치
    if len(set(labels)) > 1:
        score = silhouette_score(X, labels)
    else:
        score = -1  # 무의미한 클러스터링
    return score

def objective_lof(trial, X):
    n_neighbors = trial.suggest_int('n_neighbors', 10, 50)
    contamination = trial.suggest_float('contamination', 0.001, 0.02)

    model = LocalOutlierFactor(n_neighbors = n_neighbors,
                               contamination = comtamination,
                               novelty = True)

    model.fit(X)
    labels = pd.Series(model.predict(X)).map({1: 0, -1: 1})
    if len(set(labels)) > 1:
        score = silhouette_score(X, labels)
    else:
        score = -1
    return score

def objective_svm(trial, X):
    kernel = trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    nu = trial.suggest_float('nu', 0.001, 0.02)

    model = OneClassSVM(kernel = kernel,
                        gamma = gamma,
                        nu = nu)
    model.fit(X)
    labels = pd.Series(model.predict(X)).map({1: 0, -1: 1})
    if len(set(labels)) > 1:
        score = silhouette_score(X, labels)
    else:
        score = -1
    return score

def tune_model(objective, X, n_trials=50):
    study = optuna.create_study(direction = 'maximize')
    study.optimize(lambda trial: objective(trial, X), n_trials = n_trials)
    logger.info(f"{objective.__name__} 최적화 완료: Best Params: {study.best_params}, Best Silhouette Score: {study.best_value}")
    return study.best_params

### 설명:
- **objective_iso**: Isolation Forest 모델의 하이퍼파라미터(n_estimators, contamination, max_samples)를 최적화하여 최적의 Silhouette Score를 도출함
- **objective_lof**: Local Outlier Factor 모델의 하이퍼파라미터(n_neighbors, contamination)를 최적화함
- **objective_svm**: One-Class SVM 모델의 하이퍼파라미터(kernel, gamma, nu)를 최적화함
- **tune_model**: 주어진 objective 함수를 기반으로 Optuna를 사용하여 하이퍼파라미터 최적화를 수행

## 7. 다양한 모델 학습 및 이상치 탐지

> 이상치 탐지를 위해 다양한 비지도 학습 모델을 학습시키고, 각 모델의 예측 결과를 기반으로 이상치를 탐지.   
> 여기서는 **Autoencoder, Variational Autoencoder (VAE), Generative Adversarial Networks (GANs), Deep Belief Networks (DBN), LSTM Autoencoder, Transformer Autoencoder** 등을 다룸

### a. Autoencoder

> Autoencoder는 입력 데이터를 압축하여 잠재 공간(latent space)에 표현한 후, 이를 다시 복원하는 신경망
> 재구성 오류를 기반으로 이상치를 탐지할 수 있음

In [11]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim = 32):
        super(Autoencoder, self).__init__()

        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, latent_dim),
            nn.ReLU(True)
        )

        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

def train_autoencoder_model(X_train, X_val, input_dim, latent_dim=32, num_epochs=50, batch_size=1024):
    """
    Autoencoder 모델 학습 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - input_dim: 입력 특징의 차원 수
    - latent_dim: 잠재 공간의 차원 수
    - num_epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("Autoencoder 모델 학습 시작")

        # 텐서 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

        # DataLoader 생성
        train_dataset = TensorDataset(X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor)
        train_loader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0)
        val_loader = DataLoader(val_dataset,
                                batch_size=batch_size, 
                                shuffle=False,
                                num_workers=0)

        # 모델 초기화
        autoencoder = Autoencoder(input_dim = input_dim, latent_dim = latent_dim).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3, weight_decay=1e-5)

        best_val_loss = np.inf
        for epoch in range(num_epochs):
            autoencoder.train()
            running_loss = 0.0

            for data in train_loader:
                inputs = data[0].to(device)
                outputs = autoencoder(inputs)
                loss = criterion(outputs, inputs)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            epoch_train_loss = running_loss / len(train_loader.dataset)

            # 검증
            autoencoder.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for data in val_loader:
                    inputs = data[0].to(device)
                    outputs = autoencoder(inputs)
                    loss = criterion(outputs, inputs)
                    running_val_loss += loss.item()
            epoch_val_loss = running_val_loss / len(val_loader.dataset)

            logger.info(f"Epoch [{epoch + 1} / {num_epochs}], Train_Loss: {epoch_train_loss: .6f}, Val_Loss: {epoch_val_loss: .6f}")

            # 최적 모델 저장
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch().save(autoencoder.state_dict(), './Models/autoencoder.pth')
                logger.info(f"최적 Autoencoder 모델 저장 (Epoch {epoch + 1})")

        # 학습 완료 후 최적 모델 로드
        autoencoder.load_state_dict(torch.load('./Models/autoencoder.pth'))
        autoencoder.eval()
        logger.info("Autoencoder 모델 학습 완료 및 최적 모델 로드")

        # 재구성 오류 계산
        reconstruction_errors = []
        data_loader_full = DataLoader(TensorDataset(torch.tensor(X_full, dtype=torch.float32)),
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=0)
        logger.info("Autoencoder를 사용한 재구성 오류 계산 시작")
        for data in tqdm(data_loader_full, desc="Autoencoder Reconstruction Error"):
            inputs = data[0].to(device)
            with torch.no_grad():
                outputs = autoencoder(inputs)
                loss = torch.mean((outputs - inputs) ** 2, dim=1)  # 샘플별 MSE
                reconstruction_errors.extend(loss.cpu().numpy())

        # 재구성 오류 기반 이상치 탐지
        threshold = np.percentile(reconstruction_errors, 99.5)  # 상위 0.5%를 이상치로 간주
        anomalies = np.array(reconstruction_errors) > threshold
        logger.info("Autoencoder 기반 이상치 탐지 완료")

        return anomalies, reconstruction_errors
    except Exception as e:
        logger.error(f"Autoencoder 모델 학습 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **Autoencoder 클래스**: 인코더와 디코더로 구성된 단순한 구조의 Autoencoder를 정의
- **train_autoencoder_model 함수**: Autoencoder 모델을 학습시키고, 검증 데이터를 기반으로 최적의 모델을 저장함. 학습이 완료된 후, 전체 데이터에 대한 재구성 오류를 계산하여 이상치를 탐지

### b.Variational Autoencoder (VAE)

> VAE는 Autoencoder의 확장으로, 잠재 공간의 분포를 정규화하여 생성 모델의 특성을 가짐

In [13]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=32):
        super(VAE, self).__init__()

        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True)
        )
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)

        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

        def encoder(self, x):
            h = self.encoder(x)
            mu = self.fc_mu(h)
            logvar = self.fc_logvar(h)
            return mu, logvar

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std

        def decode(self, z):
            return self.decoder(z)

        def forward(self, x):
            mu, logvar = self.encoder(x)
            z = self.reparameterize(mu, logvar)
            reconstructed = self.decode(z)
            return reconstructed, mu, logvar

def train_vae_model(X_train, X_val, input_dim, latent_dim=32, num_epochs=50, batch_size=1024):
    """
    VAE 모델 학습 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - input_dim: 입력 특징의 차원 수
    - latent_dim: 잠재 공간의 차원 수
    - num_epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("VAE 모델 학습 시작")

        # 텐서 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

        # DataLoader 생성
        train_dataset = TensorDataset(X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor)
        train_loader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0)
        val_loader = DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=0)

        # 모델 초기화
        vae = VAE(input_dim=input_dim, latent_dim=latent_dim).to(device)
        criterion = nn.MSELoss(reduction='sum')
        optimizer = optim.Adam(vae.parameters(),
                               lr=1e-3,
                               weight_decay=1e-5)

        best_val_loss = np.inf
        for epoch in range(num_epochs):
            vae.train()
            running_loss = 0.0
            for data in train_loader:
                inputs = data[0].to(device)
                reconstructed, mu, logvar = vae(inputs)
                recon_loss = criterion(reconstructed, inputs)
                kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                loss = recon_loss + kl_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            epoch_train_loss = running_loss / len(train_loader.dataset)

            # 검증
            vae.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for data in val_loader:
                    inputs = data[0].to(device)
                    reconstructed, mu, logvar = vae(inputs)
                    recon_loss = criterion(reconstructed, inputs)
                    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                    loss = recon_loss + kl_loss
                    running_val_loss += loss.item()
            epoch_val_loss = running_val_loss / len(val_loader.dataset)

            logger.info(f"Epoch [{epoch + 1} / {num_epochs}], Train_Loss: {epoch_train_loss:.6f}, Val_Loss: {epoch_val_loss:.6f}")

            # 최적 모델 저장
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch.save(vae.state_dict(), './Modles/vae.pth')
                logger.info(f"최적 VAE 모델 저장 (Epoch {epoch + 1})")

        # 학습 완료 후 최적 모델 로드
        vae.load_state_dict(torch.load('./Models/vae.pth'))
        vae.eval()
        logger.info("VAE 모델 학습 완료 및 최적 모델 로드")

        # 재구성 오류 계산
        reconstruction_errors = []
        data_loader_full = DataLoader(TensorDataset(torch.tensor(X_full, dtype=torch.float32)),
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_worker=0)
        logger.info("VAE를 사용한 재구성 오류 계산 시작")
        for data in tqdm(data_loader_full, desc="VAE Reconstruction Error"):
            inputs = data[0].to(device)
            with torch.no_grad():
                reconstructed, mu, logvar = vae(inputs)
                loss = torch.mean((reconstructed - inputs) ** 2, dim=1)  # 샘플별 MSE
                reconstruction_errors.extend(loss.cpu().numpy())

        # 재구성 오류 기반 이상치 탐지
        threshold = np.percentile(reconstruction_errors, 99.5)  # 상위 0.5%를 이상치로 간주
        anomalies = np.array(reconstruction_errors) > threshold
        logger.info("VAE 기반 이상치 탐지 완료")

        return anomalies, reconstruction_errors
    except Exception as e:
        logger.error(f"VAE 모델 학습 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **VAE 클래스**: 인코더, 디코더, 그리고 잠재 공간의 평균과 로그 분산을 계산하는 레이어로 구성
- **train_vae_model 함수**: VAE 모델을 학습시키고, 검증 데이터를 기반으로 최적의 모델을 저장. 학습 완료 후, 전체 데이터에 대한 재구성 오률를 계산하여 이상치를 탐지함

### c. Generative Adversarial Networks (GANs)

> GANs는 생성자(Generator)와 판별자(Discriminator)로 구성된 모델로, 데이터의 분포를 학습하여 새로운 데이터를 생성할 수 있음.   
> 이상치 탐지에서는 판별자의 출력 확률을 기반으로 이상치를 식별함.

In [15]:
class Generator(nn.Module):
    def __init__(self, input_dim, latent_dim=32):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

def train_gan(generator, discriminator, dataloader, optimizer_G, optimizer_D, criterion, device, num_epochs=50):
    """
    GANs 학습 함수
    - generator: 생성자 모델
    - discriminator: 판별자 모델
    - dataloader: 데이터 로더
    - optimizer_G: 생성자 최적화 알고리즘
    - optimizer_D: 판별자 최적화 알고리즘
    - criterion: 손실 함수 (예: BCELoss)
    - device: 학습 디바이스
    - num_epochs: 학습 에폭 수
    """
    try:
        generator.train()
        discriminator.train()

        for epoch in range(num_epochs):
            for data in dataloader:
                real = data[0].to(device)
                batch_size = real.size(0)

                # 진짜 데이터 레이블: 1, 가짜 데이터 레이블: 0
                real_labels = torch.ones(batch_size, 1).to(device)
                fake_labels = torch.zeros(batch_size, 1).to(device)

                # 생성자 학습
                optimizer_G.zero_grad()
                z = torch.randn(batch_size, generator.model[0].in_features).to(device)
                fake = generator(z)
                outputs = discriminator(fake)
                loss_G = criterion(outputs, real_labels)
                loss_G.backward()
                optimizer_G.step()

                # 판별자 학습
                optimizer_D.zero_grad()
                outputs_real = discriminator(real)
                loss_D_real = criterion(outputs_real, real_labels)
                outputs_fake = discriminator(fake.detach())
                loss_D_fake = criterion(outputs_fake, fake_labels)
                loss_D = loss_D_real + loss_D_fake
                loss_D.backward()
                optimizer_D.step()

            logger.info(f"Epoch [{epoch + 1} / {num_epochs}], Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")
            
        return generator, discriminator
    except Exception as e:
        logger.error(f"GANs 학습 중 오류 발생: {e}")
        return None, None

def detect_anomalies_gan(generator, discriminator, X, device, threshold=0.5):
    """
    GANs를 사용한 이상치 탐지 함수
    - generator: 학습된 생성자 모델
    - discriminator: 학습된 판별자 모델
    - X: 전체 데이터 (numpy array)
    - device: 학습 디바이스
    - threshold: 이상치로 간주할 판별자 출력 확률 임계값
    """
    try:
        generator.eval()
        discriminator.eval()

        with torch.no_grad():
            z = torch.randn(X.shape[0], generator.model[0].in_features).to(device)
            fake = generator(z)
            outputs = discriminator(fake)
            # 판별자가 가짜 데이터에 대해 출력한 확률을 이상치 점수로 사용
            anomaly_scores = 1 - outputs.cpu().numpy().flatten()
        # 임계값을 기준으로 이상치 판별
        anomalies = anomaly_scores > threshold
        return anomalies, anomaly_score
    except Exception as e:
        logger.error(f"GANs를 사용한 이상치 탐지 중 오류 발생: {e}")
        return None, None

def train_and_detect_gans(X_train, X_val, X_full, input_dim, latent_dim=32, num_epochs=50, batch_size=1024):
    """
    GANs 모델 학습 및 이상치 탐지 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - X_full: 전체 데이터 (numpy array)
    - input_dim: 입력 특징의 차원 수
    - latent_dim: 잠재 공간의 차원 수
    - num_epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("GANs 모델 학습 및 예측 시작")

        #텐서 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

        # DataLoader 생성
        train_dataset = TensorDataset(X_train_tensor)
        train_loader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0)

        # 모델 초기화
        generator = Generator(input_dim=input_dim, latent_dim=latent_dim).to(device)
        discriminator = Discriminator(input_dim=input_dim).to(device)

        # 손실 함수 및 최적화 알고리즘 설정
        criterion = nn.BCELoss()
        optimizer_G = optim.Adam(generator.parameters(),
                                 lr=1e-3,
                                 weight_decay=1e-5)
        optimizer_D = optim.Adam(discriminator.parameters(),
                                 lr=1e-3,
                                 weight_decay=1e-5)

        # GAN 학습
        generator, discriminator = train_gan(generator,
                                             discriminator,
                                             train_loader,
                                             optimizer_G,
                                             optimizer_D,
                                             criterion,
                                             device,
                                             num_epochs=num_epochs)
        logger.info("GANs 학습 완료")

        # 이상치 탐지
        logger.info("GANs 기반 이상치 탐지 시작")
        anomalies_gan, anomaly_scores_gan = detect_anomalies_gan(generator,
                                                                 discriminator,
                                                                 X_full,
                                                                 device,
                                                                 threshold=0.5)
        logger.info("GANs 기반 이상치 탐지 완료")

        return anomalies_gan, anomaly_scores_gan
    except Exception as e:
        logger.info(f"GANs 학습 및 이상치 탐지 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **Generator 및 Discriminator 클래스**: GANs 의 생성자와 판별자를 정의
- **train_gan 함수**: GANs 모델을 학습시키는 함수로, 생성자와 판별자를 번갈아가며 학습시킴
- **detect_anomalies_gan 함수**: 학습된 GANs를 사용하여 이상치를 탐지. 판별자의 출력 확률을 기반으로 이상치를 식별함
- **train_and_detect_gan 함수**: 전체 과정을 통합하여 GANs 모델을 학습시키고 이상치를 탐지함

### d. Deep Belief Networks (DBN)

> DBN은 여러 개의 Restricted Boltzmann Machines (RBM)으로 구성된 신경망으로, 비지도 학습을 통해 특징을 학습함

In [16]:
class RBM(nn.Module):
    def __init__(self, n_vis, n_hid):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hid, n_vis) * 0.01)
        self.h_bias = nn.Parameter(torch.zeros(n_hid))
        self.v_bias = nn.Parameter(torch.zeros(n_vis))

    def sample_h(self, v):
        wx = torch.matmul(v, self.W.t()) + self.h_bias
        p_h_given_v = torch.sigmoid(wx)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, h):
        wx = torch.matmul(h, self.W) + self.v_bias
        p_v_given_h = torch.sigmoid(wx)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def forward(self, v):
        p_h, h = self.sample_h(v)
        p_v, v = self.sample_v(h)
        return v

def train_rbm(rbm, train_loader, lr=0.01, k=1, epochs=10):
    """
    RBM 학습 함수
    - rbm: RBM 모델
    - train_loader: 학습 데이터 로더
    - lr: 학습률
    - k: Gibbs 샘플링 단계 수
    - epochs: 학습 에폭 수
    """
    try:
        optimizer = optim.SGD(rbm.parameters(), lr=lr)
        for epoch in range(epochs):
            loss_ = 0.0
            for data in train_loader:
                v0 = data[0].to(device)
                vk = v0
                for _ in range(k):
                    p_h, h = rbm.sample_h(vk)
                    p_v, vk = rbm.sample_v(h)
                loss =  torch.mean((v0 - vk) ** 2)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_ += loss.item()
            logger.info(f"RBM Epoch [{epoch + 1} / {epochs}], Loss: {loss_ / len(train_loader):.6f}")
        return rbm
    except Exception as e:
        logger.error(f"RBM 학습 중 오류 발생: {e}")
        return rbm

class DBN(nn.Module):
    def __init__(self, layers):
        super(DBN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i + 1]))
            self.layers.append(nn.ReLU(True))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def train_dbn_model(X_train, X_val, layers=[128, 64], lr=0.01, epochs=50, batch_size=1024):
    """
    DBN 모델 학습 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - layers: 각 레이어의 노드 수 리스트
    - lr: 학습률
    - epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("DBN 모델 학습 시작")

        # RBM 초기화 및 학습
        rbm = RBM(n_vis=X_train.shape[1], n_hid=layers[0]).to(device)
        train_loader_rbm = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32)),
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=0)
        rbm = train_rbm(rbm, train_loader_rbm, lr=lr, k=1, epochs=10)

        # DBN 초기화
        dbn = DBN(layers=[X_train.shape[1]] + layers).to(device)
        dbn.eval()

        # DBN을 사용한 특징 추출 (전이 학습)
        with torch.no_grad():
            X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            X_train_encoded = rbm.encode(X_train_tensor)[0]
            X_val_encoded = rbm.encode(X_val_tensor)[0]
            X_train_dbn = dbn(X_train_encoded).cpu().numpy()
            X_val_dbn = dbn(X_val_encoded).cpu().numpy()

        # 이상치 탐지 (재구성 오류 기반)
        criterion = nn.MSELoss(reduction='sum')
        X_train_tensor_dbn = torch.tensor(X_train_dbn, dtype=torch.float32).to(device)
        X_val_tensor_dbn = torch.tensor(X_val_dbn, dtype=torch.float32).to(device)
        reconstructed = dbn(X_train_tensor_dbn)
        loss = criterion(reconstructed, X_train_tensor_dbn)
        logger.info(f"DBN 재구성 손실: {loss.item():.6f}")

        # 재구성 오류 기반 이상치 탐지
        reconstruction_errors_dbn = np.mean((X_train_dbn - dbn(torch.tensor(X_train_dbn, dtype=torch.float32).to(device)).cpu().numpy()) ** 2, axis=1)
        threshold = np.percentile(reconstruction_errors_dbn, 99.5)
        anomalies_dbn = reconstruction_errors_dbn > threshold
        logger.info("DBN 기반 이상치 탐지 완료")

        return anomalies_dbn, reconstruction_errors_dbn
    except Exception as e:
        logger.error(f"DBN 모델 학습 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **RBM 클래스**: Restricted Boltzmann Machine을 정의하며, 인코더와 디코더 역할을 수행함
- **train_rbm 함수**: RBM 모델을 학습시키는 함수로, Contrastive Divergence 알고리즘을 사용하여 학습함
- **DBN 클래스**: 여러 개의 RBM으로 구성된 Deep Belief Network를 정의함
- **train_dbn_model 함수**: DBn 모델을 학습시키고, 재구성 오류를 기반으로 이상치를 탐지함

### e. LSTM Autoencoder

> LSTM Autoencoder는 시계열 데이터의 시간적 패턴을 학습하는 데 특화된 Autoencoder로, 재구성 오류를 기반으로 이상치를 탐지함.

In [17]:
class LSTM_Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, latent_dim=32):
        super(LSTM_Autoencoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # 인코더
        self.lstm_enc = nn.LSTM(input_dim,
                                hidden_dim,
                                num_layers,
                                batch_first=True)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

        # 디코더
        self.fc_dec = nn.Linear(latent_dim, hidden_dim)
        self.lstm_dec = nn.LSTM(hidden_dim,
                                input_dim,
                                num_layers,
                                batch_first=True)

    def encode(self, x):
        h, _ = self.lstm_enc(x)
        h = h[:, -1, :]  # 마지막 타임스텝의 출력
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, seq_len):
        h = self.fc_dec(z).unsqueeze(1).repeat(1, seq_len, 1)
        reconstructed, _ = self.lstm_dec(h)
        return reconstructed

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z, x.size(1))
        return reconstructed, mu, logvar

def train_lstm_ae_model(X_train, X_val, X_full, input_dim, hidden_dim=64, num_layers=2, latent_dim=32, num_epochs=50, batch_size=1024):
    """
    LSTM Autoencoder 모델 학습 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - X_full: 전체 데이터 (numpy array)
    - input_dim: 입력 특징의 차원 수
    - hidden_dim: LSTM의 숨겨진 상태 차원
    - num_layers: LSTM 레이어 수
    - latent_dim: 잠재 공간의 차원 수
    - num_epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("LSTM Autoencoder 모델 학습 시작")

        # 텐서 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

        # DataLoader 생성
        train_dataset = TensorDataset(X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor)
        train_loader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0)
        val_loader = DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=0)

        # 모델 초기화
        lstm_ae = LSTM_Autoencoder(input_dim=input_dim,
                                   hidden_dim=hidden_dim,
                                   num_layers=num_layers,
                                   latent_dim=latent_dim).to(device)
        criterion = nn.MSELoss(reduction='sum')
        optimizer = optim.Adam(lstm_ae.parameters(),
                               lr=1e-3,
                               weight_decay=1e-5)

        best_val_loss = np.inf
        for epoch in range(num_epochs):
            lstm_ae.train()
            running_loss = 0.0
            for data in train_loader:
                inputs = data[0].to(device)
                reconstructed, mu, logvar = lstm_ae(inputs)
                recon_loss = criterion(reconstructed, inputs)
                kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                loss = recon_loss + kl_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            epoch_train_loss = running_loss / len(train_loader.dataset)

            # 검증
            lstm_ae.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for data in val_loader:
                    inputs = data[0].to(device)
                    reconstructed, mu, logvar = lstm_ae(inputs)
                    recon_loss = criterion(reconstructed, inputs)
                    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                    loss = recon_loss + kl_loss
                    running_val_loss += loss.item()
            epoch_val_loss = running_val_loss / len(val_loader.dataset)

            logger.info(f"Epoch [{epoch + 1} / {num_epochs}], Train_Loss: {epoch_train_loss:.6f}, Val_Loss: {epoch_val_loss:.6f}")

            # 최적 모델 저장
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch.save(lstm_ae.state_dict(), './Models/lstm_ae.pth')
                logger.info(f"최적 LSTM Autoencoder 모델 저장 (Epoch {epoch + 1})")

        # 학습 완료 후 최적 모델 로드
        lstm_ae.load_state_dict(torch.load('./Models/lstm_ae.pth'))
        lstm_ae.eval()
        logger.info("LSTM Autoencoder 모델 학습 완료 및 최적 모델 로드")

        # 재구성 오류 계산
        reconstruction_errors = []
        data_loader_full = DataLoader(TensorDataset(torch.tensor(X_full, dtype=torch.float32)),
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=0)
        logger.info("LSTM Autoencoder를 사용한 재구성 오류 계산 시작")
        for data in tqdm(data_loader_full, desc="LSTM AE Reconstruction Error"):
            inputs = data[0].to(device)
            with torch.no_grad():
                reconstructed, mu, logvar = lstm_ae(inputs)
                loss = torch.mean((reconstructed - inputs) ** 2, dim=(1, 2))  # 샘플별 MSE
                reconstruction_errors.extend(loss.cpu().numpy())

        # 재구성 오류 기반 이상치 탐지
        threshold = np.percentile(reconstruction_errors, 99.5)  # 상위 0.5%를 이상치로 간주
        anomalies = np.array(reconstruction_errors) > threshold
        logger.info("LSTM Autoencoder 기반 이상치 탐지 완료")

        return anomalies, reconstruction_errors
    except Exception as e:
        logger.error(f"LSTM Autoencoder 모델 학습 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **LSTM_Autoencoder 클래스**: LSTM을 인코더와 디코더로 사용하여 시계열 데이터를 학습함
- **train_lstm_ae_model 함수**: LSTM Autoencoder 모델을 학습시키고, 재구성 오류를 기반으로 이상치를 탐지함

### f. Transformer Autoencoder

> Transformer 기반 Autoencoder는 시계열 데이터의 장기적인 의존성을 학습하는 데 유리한 구조

In [18]:
class Transformer_Autoencoder(nn.Module):
    def __init__(self, input_dim, seq_length, nhead=8, num_layers=3, dim_feedforward=512, latent_dim=32):
        super(Transformer_Autoencoder, self).__init__()
        self.seq_length = seq_length
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim,
                                                        nhead=nhead,
                                                        dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer,
                                                         num_layers=num_layers)
        self.fc_mu = nn.Linear(input_dim * seq_length, latent_dim)
        self.fc_logvar = nn.Linear(input_dim * seq_length, latent_dim)
        self.fc_dec = nn.Linear(latent_dim, input_dim * seq_length)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=input_dim,
                                                        nhead=nhead,
                                                        dim_feedforward=dim_feed_forward)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer,
                                                         num_layers=num_layers)

    def encode(self, x):
        # x: (batch, seq_length, input_dim)
        x = x.permute(1, 0, 2)  # (seq_length, batch, input_dim)
        encoded = self.transformer_encoder(x)  # (seq_length, batch, input_dim)
        encoded = encoded.permute(1, 0, 2).contiguous().view(x.size(1), -1)  # (batch, seq_length * input_dim)
        mu = self.fc_mu(encoded)
        logvar = self.fc_logvar(encoded)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        decoded = self.fc_dec(z)  # (batch, seq_length * input_dim)
        decoded = decoded.view(-1, self.seq_length, self.input_dim)  # (batch, seq_length, input_dim)
        decoded = decoded.permute(1, 0, 2)  # (seq_length, batch, input_dim)
        decoded = self.transformer_decoder(decoded, memory=None)  # (seq_length, batch, input_dim)
        decoded = decoded.permute(1, 0, 2).contiguous()  # (batch, seq_length, input_dim)
        return decoded

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar

def train_transformer_ae_model(X_train, X_val, X_full, input_dim, seq_length, nhead=8, num_layers=3, dim_feedforward=512, latent_dim=32, num_epochs=50, batch_size=1024):
    """
    Transformer Autoencoder 모델 학습 함수
    - X_train: 학습 데이터 (numpy array)
    - X_val: 검증 데이터 (numpy array)
    - X_full: 전체 데이터 (numpy array)
    - input_dim: 입력 특징의 차원 수
    - seq_length: 시계열 길이
    - nhead: Multi-Head Attention의 헤드 수
    - num_layers: Transformer 레이어 수
    - dim_feedforward: Feedforward 신경망의 차원 수
    - latent_dim: 잠재 공간의 차원 수
    - num_epochs: 학습 에폭 수
    - batch_size: 배치 크기
    """
    try:
        logger.info("Transformer Autoencoder 모델 학습 시작")

        # 텐서 변환 및 시퀸스 길이 맞추기
        X_train_seq = X_train.reshape(-1,
                                      seq_length,
                                      input_dim)
        X_val_seq = X_val.reshape(-1,
                                  seq_length,
                                  input_dim)
        X_full_seq = X_full.reshape(-1,
                                    seq_length,
                                    input_dim)

        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32)

        # DataLoader 생성
        train_dataset = TensorDataset(X_train_tensor)
        val_dataset = TensorDataset(X_val_tensor)
        train_loader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0)
        val_loader = DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=0)

        # 모델 초기화
        transformer_ae = Transformer_Autoencoder(input_dim=input_dim,
                                                 seq_length=seq_length,
                                                 num_layers=num_layers,
                                                 dim_feedforward=dim_feedforward,
                                                 latent_dim=latent_dim).to(device)
        criterion = nn.MSELoss(reduction='sum')
        optimizer = optim.Adam(transformer_ae.parameters(),
                               lr=1e-3,
                               weight_decay=1e-5)

        best_val_loss = np.inf
        for epoch in range(num_epochs):
            transformer_ae.train()
            running_loss = 0.0
            for data in train_loader:
                inputs = data[0].to(device)
                reconstructed, mu, logvar = transformer_ae(inputs)
                recon_loss = criterion(reconstructed, inputs)
                kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                loss = recon_loss + kl_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            epoch_train_loss = running_loss / len(train_loader.dataset)

            # 검증
            transformer_ae.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for data in val_loader:
                    inputs = data[0].to(device)
                    reconstructed, mu, logvar = transformer_ae(inputs)
                    recon_loss = criterion(reconstructed, inputs)
                    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
                    loss = recon_loss + kl_loss
                    running_val_loss += loss.item()
            epoch_val_loss = running_val_loss / len(val_loader.dataset)

            logger.info(f"Epoch [{epoch + 1} / {num_epochs}], Train_Loss: {epoch_train_loss:.6f}, Val_Loss: {epoch_val_loss:.6f}")

            # 최적 모델 저장
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch.save(transformer_ae.state_dict(), './Models/transformer_ae.pth')
                logger.info(f"최적 Transformer Autoencoder 모델 저장 (Epoch {epoch + 1})")

        # 학습 완료 후 최적 모델 로드
        transformer_ae.load_state_dict(torch.load('./Models/transformer_ae.pth'))
        transformer_ae.eval()
        logger.info("Transformer Autoencoder 모델 학습 완료 및 최적 모델 로드")

        # 재구성 오류 계산
        reconstruction_errors = []
        data_loader_full = DataLoader(TensorDataset(torch.tensor(X_full_seq, dtype=torch.float32)),
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=0)
        logger.info("Transformer Autoencoder를 사용한 재구성 오류 계산 시작")
        for data in tqdm(data_loader_full, desc="Transformer AE Reconstruction Error"):
            inputs = data[0].to(device)
            with torch.no_grad():
                reconstructed, mu, logvar = transformer_ae(inputs)
                loss = torch.mean((reconstructed - inputs) ** 2, dim=(1, 2))  # 샘플별 MSE
                reconstruction_errors.extend(loss.cpu().numpy())

        # 재구성 오류 기반 이상치 탐지
        threshold = np.percentile(reconstruction_errors, 99.5)  # 상위 0.5%를 이상치로 간주
        anomalies = np.array(reconstruction_errors) > threshold
        logger.info("Transformer Autoencoder 기반 이상치 탐지 완료")

        return anomalies, reconstruction_errors
    except Exception as e:
        logger.error(f"Transformer Autoencoder 모델 학습 중 오류 발생: {e}")
        sys.exit(1)

### 설명:
- **Transformer_Autoencoder 클래스**: Transformer 인코더와 디코더를 사용하여 시계열 데이터를 학습함
- **train_transformer_ae_model 함수**: Transformer Autoencoder 모델을 학습시키고, 재구성 오류를 기반으로 이상치를 탐지함

***
## 8. 앙상블 방법: Majority Voting

> 여러 모델의 예측을 결합하여 최종 이상치를 판별하기 위해 앙상블 방법 중 **Majority Voting**을 도입.   
> 이는 비지도 학습 모델의 예측 결과를 통합하여 보다 신뢰성 있는 이상치 탐지를 가능하게 함

In [2]:
def majority_voting(anomalies_list, threshold=3):
    """
    Majority Voting을 사용하여 앙상블을 수행함.
    - anomalies_list: 각 모델의 이상치 예측 결과 리스트 (numpy array)
    - threshold: 이상치로 간주할 최소 모델 수
    """
    try:
        logger.info("Majority Voting을 사용한 앙상블 시작")
        # 각 모델의 이상치 예측을 합산
        combined = np.sum(anomalies_list, axis=0)
        # threshold 이상일 때 이상치로 판별
        final_anomalies = combined >= threshold
        logger.info("Majority Voting 앙상블 완료")
        return final_anomalies
    except Exception as e:
        logger.error(f"Majority Voting 앙상블 중 오류 발생: {e}")
        return None

### 설명:
- **anomalies_list**: 각 모델이 예측한 이상치의 리스트를 전달함
- **threshold**: 여러 모델 중 최소 몇 개의 모델이 이상치로 예측해야 최종적으로 이상치로 판별할 지를 설정함
- **최종 이상치 판별**: 각 샘플에 대해 이상치로 예측한 모델의 수가 threshold 이상인 경우 이상치로 판별함

***
## 9. 모델 평가

> 모델의 성능을 평가하기 위해 다양한 지표를 계산함.   
> 여기서는 Silhouette Score와 같은 클러스터링 지표를 사용하여 모델의 품질을 평가함.

In [ ]:
def evaluate_models(anomalies, X):
    """
    모델 평가 지표를 계산함.
    - anomalies: 이상치 예측 결과 (numpy array)
    - X: 입력 데이터 (numpy array)
    """
    try:
        logger.info("모델 평가 지표 계산 시작")

        # 클러스터링을 위한 라벨 생성 (0: 정상, 1: 이상치)
        labels = anomalies.astype(int)

        # Silhouette Score 계산
        if len(set(labels)) > 1:
            silhouette = silhouette_score(X, labels)
            davies_bouldin = davies_bouldin_score(X, labels)
            calinski_harabasz = calinski_harabasz_score(X, labels)
        else:
            silhouette = -1
            davies_bouldin = -1
            calinski_harabasz = -1

        evaluation_metrics = {
            'Silhouette Score': silhouette,
            'Davies-Bouldin Score': davies_bouldin,
            'Calinski-Harabasz Score': calinski_harabasz
        }

        logger.info(f"Silhouette Score: {silhouette}")
        logger.info(f"Davies-Bouldin Score: {davies_bouldin}")
        logger.info(f"Calinski-Harabasz Score: {calinski_harabasz}")

        return evaluation_metrics
    except Exception as e:
        logger.error(f"모델 평가 지표 계산 중 오류 발생: {e}")
        return None

### 설명:
- **Silhouette Score**: 클러스터의 응집도와 분리도를 평가하는 지표로, 1에 가까울수록 좋은 클러스터링을 의미함
- **Davies-Bouldin Score**: 클러스터 간의 분리를 평가하는 지표로, 값이 낮을수록 좋은 클러스터링을 의미함
- **Calinski-Harabasz Score**: 클러스터 내 응집도와 클러스터 간 분리를 평가하는 지표로, 값이 높을수록 좋은 클러스터링을 의미함

***
## 10.모델 해석 및 투명성 강화

> 모델의 예측을 해석하고, 이상치 탐지의 원인을 설명할 수 있도록 **SHAP**과 **LIME**을 활용하여 모델의 투명성을 강화함

### a. SHAP를 사용한 모델 해석
> **SHAP**(SHapley Additive exPlanations)는 모델의 예측에 기여한 각 특징의 중요도를 시각화하는 도구임

In [ ]:
def interpret_model_shap(model, X, feature_names):
    """
    SHAP를 사용하여 모델 해석
    - model: 학습된 모델
    - X: 특징 데이터 (numpy array 또는 pandas DataFrame)
    - feature_names: 특징 이름 리스트
    """
    try:
        logger.info("SHAP를 사용한 모델 해석 시작")
        explainer = shap.Explainer(model, X)
        shap_values = explainer(X)
        shap.summary_plot(shap_values, features=X, feature_names=feature_names)
        logger.info("SHAP 모델 해석 완료")
    except Exception as e:
        logger.error(f"SHAP 모델 해석 중 오류 발생: {e}")

### 설명:
- **Explainer 생성**: 모델과 데이터를 기반으로 SHAP Explainer를 생성함.
- **Shap Values 계산**: 각 샘플에 대한 Shapley 값을 계산함.
- **시각화**: SHAP summary plot을 통해 특징의 중요도를 시각화함.

### b. LIME을 사용한 모델 해석
> **LIME**(Local Interpretable Model-agnostic Explanations)은 특정 샘플에 대한 모델의 예측을 해석하는 도구임.

In [ ]:
def interpret_model_lime(model, X, feature_names, num_features=10):
    """
    LIME을 사용하여 모델 해석
    - model: 학습된 모델
    - X: 특징 데이터 (numpy array)
    - feature_names: 특징 이름 리스트
    - nu_features: 해석할 특징 수
    """
    try:
        logger.info("LIME을 사용한 모델 해석 시작")
        explainer = lime_tabular.LimeTabularExplainer(
            training_data = X,
            feature_names = feature_names,
            mode = 'classification',
            discretize_continuous=True
        )
        # 예시로 첫 번째 샘플을 해석
        i = 0
        exp = explainer.explain_instance(X[i],
                                         model.predict_proba,
                                         num_features=num_features)
        exp.show_in_notebook(show_table=True)
        logger.info("LIME 모델 해석 완료")
    except Exception as e:
        logger.error(f"LIME 모델 해석 중 오류 발생: {e}")

### 설명:
- **Explainer 생성**: LIME Explainer를 생성하여 모델의 로컬 해석을 준비함.
- **설명 생성**: 특정 샘플에 대한 모델의 예측을 해석함.
- **시각화**: LIME explanation을 통해 주요 특징의 기여도를 시각화함.

***
## 11. 모델 최적화 및 경량화

> 모델의 추론 속도를 최적화하고, 리소스 효율적인 모델로 경량화하기 위해 **ONNX**와 **TensorRT**를 사용함.

### a. PyTorch 모델을 ONNX로 변환

> ONNX(Open Neural Network Exchange)는 다양한 딥러닝 프레임워크 간의 모델 호환성을 제공함.

In [ ]:
def convert_to_onnx(model, input_sample, model_name='model'):
    """
    PyTorch 모델을 ONNX 형식으로 변환함.
    - model: 학습된 PyTorch 모델
    - input_sample: 모델에 대한 입력 샘플 (numpy array 또는 torch tensor)
    - model_name: 저장할 ONNX 모델 파일 이름
    """
    try:
        logger.info(f"모델을 ONNX 형식으로 변환 시작: {model_name}.onnx")
        model.eval()
        if isinstance(input_sample, np.ndarray):
            input_sample = torch.from_numpy(input_sample).float()
        torch.onnx.export(model, input_sample, f"{model_name}.onnx",
                          export_params=True,
                          opset_version=12,
                          do_constant_folding=True,
                          input_names=['input'],
                          output_names=['output'],
                          dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
        logger.info(f"모델을 ONNX 형식으로 변환 완료: {model_name}.onnx")
    except Exception as e:
        logger.error(f"모델을 ONNX 형식으로 변환 중 오류 발생: {e}")

### 설명:
- **모델 변환**: PyTorch 모델을 ONNX 형식으로 변환하여 다양한 플랫폼에서 사용할 수 있게 함.
- **입력 샘플**: 변환 과정에서 필요한 입력 샘플을 제공하여 모델의 구조를 정의함.

### b. TensorRT를 사용한 모델 최적화
> TensorRT는 NVIDIA GPU에서 딥러닝 모델의 추론 속도를 최적화하는 도구.

In [ ]:
def optimize_model_with_tensrrt(onnx_model_path, trt_model_path):
    """
    TensorRT를 사용하여 ONNX 모델을 최적화함.
    - onnx_model_path: 변화된 ONNX 모델 파일 경로
    - trt_model_path: 최적화된 TensorRT 모델 파일 경로
    """
    try:
        logger.info(f"TensorRT를 사용한 모델 최적화 시작: {onnx_model_path} -> {trt_model_path}")
        TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
        builder = trt.Builder(TRT_LOGGER)
        network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
        parser = trt.OnnxParser(network, TRT_LOGGER)

        with open(onnx_model_path, 'rb') as model:
            if not parser.parse(model.read()):
                for error in range(parser.num_errors):
                    logger.error(parser.get_error(error))
                return False

        builder.max_workspace_size = 1 << 30  # 1GB
        builder.max_batch_size = 1
        engine = builder.build_cuda_engine(network)

        with open(trt_model_path, 'wb') as f:
            f.write(engine.serialize())

        logger.info(f"TensorRT를 사용한 모델 최적화 완료: {trt_model_path}")
        return True
    except Exception as e:
        logger.error(f"TensorRT를 사용한 모델 최적화 중 오류 발생: {e}")
        return False

### 설명:
- **TensorRT 최적화**: ONNX 모델을 TensorRT 엔진으로 변환하여 NVIDIA GPU에서의 추론 속도를 향상시킴.
- **설정**: 최대 워크스페이스 크기 및 배치 크기를 설정하여 최적화를 수행함.

***
## 12. 파이프라인 자동화: Airflow 활용

> 전체 파이프라인의 데이터 처리, 모델 학습, 평가, 배포 단계를 자동화하기 위해 **Apache Airflow**를 활용함.   
> Airflow는 워크플로우를 DAG(Directed Acyclic Graph) 형태로 정의하고, 각 작업을 스케줄링하며 관리할 수 있음.